**Sentiment Analysis**

**I. Individual Analysis**

**Books**

*General Imports*

In [1]:
import nltk
from random import shuffle
from sklearn.linear_model import LogisticRegression
from nltk.metrics.scores import (precision, recall, accuracy)

**Features Construction**

Useful functions to load the different features models

In [2]:
def load_bow(thefeatures,filename):
    '''
    Loads the tags in the filename and appends them to the
    features list.
    '''
    f = open(filename, "r")
    line = f.readline()
    while len(line) >0:
        splited = line.split(' ')
        feats = {}
        for word in range(len(splited)-1):
            theword = splited[word].split(":")
            feats[theword[0]] = int(theword[1])
        tag = splited[-1].split("#:")[1][0:3]
        thefeatures.append((feats,tag))
        line = f.readline()
    f.close()

In [3]:
def load_binary_bow(thefeatures,filename):
    '''
    Loads the tags in the filename and appends them to the
    features list. Filters them to change it to 1 for each 
    existent element in the Bag of Words.
    '''
    f = open(filename, "r")
    line = f.readline()
    while len(line) >0:
        splited = line.split(' ')
        feats = {}
        for word in range(len(splited)-1):
            theword = splited[word].split(":")
            feats[theword[0]] = 1
        tag = splited[-1].split("#:")[1][0:3]
        thefeatures.append((feats,tag))
        line = f.readline()
    f.close()

In [4]:
def load_lexicon(thefeatures,filename):
    '''
    Loads the tags process them to create the Features based on the
    AFINN-111 sentiment lexicon.
    '''
    lex = load_affin()
    f = open(filename, "r")
    line = f.readline()
    while len(line) >0:
        splited = line.split(' ')
        feats1 = {}
        summa = 0
        negga = 0
        pos = 0
        neg = 0
        tot = 0
        
        for word in range(len(splited)-1):
            add = False
            theword = splited[word].split(":")
            lilwords = theword[0].split("_")
            for lw in lilwords:
                if lw in lex:
                    add = True
                    if lex[lw] > 0:
                        summa += lex[lw]*int(theword[1])
                        pos += int(theword[1])
                    else:
                        negga += lex[lw]*int(theword[1])
                        neg += int(theword[1])
            tot += int(theword[1])
        feats1["point_pos"] = summa
        feats1["point_neg"] = -negga
        feats1["count_pos"] = pos
        feats1["count_neg"] = neg
        feats1["size"] = tot
        feats1["porc_pos"] = pos/tot
        feats1["porc_neg"] = neg/tot
        tag = splited[-1].split("#:")[1][0:3]
        thefeatures.append((feats1,tag))
        line = f.readline()
    f.close()

def load_affin():
    '''
    Loads the lexicon of sentiment words included in the AFINN-11 lexicon
    '''
    f = open("AFINN-111.txt", "r")
    line = f.readline()
    affin = {}
    while len(line) >0:
        splited = line.split('\t')
        affin[splited[0]] = int(splited[1])
        line = f.readline()
    return affin

*Additional Function*

Shows the most informative features for the Linear Regression Models.

Taken from the answers to: https://stackoverflow.com/questions/11116697/how-to-get-most-informative-features-for-scikit-learn-classifiers Kudos to him.

In [5]:
def show_most_informative_features(vectorizer, clf, n=20):
    '''
    Gets the most informative features from a Linear Regression Model
    '''
    feature_names = vectorizer.get_feature_names()
    coefs_with_fns = sorted(zip(clf.coef_[0], feature_names))
    top = zip(coefs_with_fns[:n], coefs_with_fns[:-(n + 1):-1])
    for (coef_1, fn_1), (coef_2, fn_2) in top:
        print ("\t%.4f\t%-15s\t\t%.4f\t%-15s" % (coef_1, fn_1, coef_2, fn_2))

**BoW**

Loads the data and creates the feature to train. Loads and uses both review files.

In [6]:
BoWFeatures = []

#Loads the positive reviews
load_bow(BoWFeatures,"reviews/books/positive.review")

#Loads the negative reviews 
load_bow(BoWFeatures,"reviews/books/negative.review")

print("Loaded",len(BoWFeatures),"reviews")

Loaded 2000 reviews


Loads the data and creates the feature to test the classifier. 

In [7]:
BoWTesting =[]

#Loads the testing reviews 
load_bow(BoWTesting,"reviews/books/unlabeled.review")

print("Loaded",len(BoWTesting),"reviews")

Loaded 4465 reviews


**Binary BoW**

Loads the data and creates the feature to train. Loads and uses both review files. Filters them to change it to 1 for each existent element in the Bag of Words.

In [8]:
BinaryBoWFeatures = []

#Loads the positive reviews
load_binary_bow(BinaryBoWFeatures,"reviews/books/positive.review")

#Loads the negative reviews 
load_binary_bow(BinaryBoWFeatures,"reviews/books/negative.review")

print("Loaded",len(BinaryBoWFeatures),"reviews")

Loaded 2000 reviews


Loads the data and creates the feature to test the classifier. 

In [9]:
BinaryBoWTesting =[]

#Loads the testing reviews 
load_binary_bow(BinaryBoWTesting,"reviews/books/unlabeled.review")

print("Loaded",len(BinaryBoWTesting),"reviews")

Loaded 4465 reviews


**Lexicon Features**

Loads the data and creates the feature to train. Loads and uses both review files. Takes into use the following Features:

- Number of positive words
- Number of negative words
- Positive score following the AFINN-111 scores
- Negative score following the AFINN-111 scores
- Size (number of words) of the review
- Percentage of positive words
- Percentage of negative words

In [10]:
LexiconFeatures = []

#Loads the positive reviews
load_lexicon(LexiconFeatures,"reviews/books/positive.review")

#Loads the negative reviews 
load_lexicon(LexiconFeatures,"reviews/books/negative.review")

print("Loaded",len(LexiconFeatures),"reviews")

Loaded 2000 reviews


Loads the data and creates the feature to test the classifier. 

In [11]:
LexiconTesting =[]

#Loads the testing reviews 
load_lexicon(LexiconTesting,"reviews/books/unlabeled.review")

print("Loaded",len(LexiconTesting),"reviews")

Loaded 4465 reviews


**Testing Zone**

Now we are gonna test the six different combinations of Features and classification algorithms

**BoW - Naive Bayes**

Trains the dataset over a Naive Bayes with a Bag of Words Model.

**Training**

In [12]:
shuffle(BoWFeatures)
classifier = nltk.NaiveBayesClassifier.train(BoWFeatures)
classifier.show_most_informative_features(10)

Most Informative Features
           disappointing = 1                 neg : pos    =     40.3 : 1.0
              your_money = 1                 neg : pos    =     27.0 : 1.0
              waste_your = 1                 neg : pos    =     20.3 : 1.0
                     bad = 2                 neg : pos    =     16.3 : 1.0
                   waste = 1                 neg : pos    =     15.4 : 1.0
             a_wonderful = 1                 pos : neg    =     15.4 : 1.0
                  a_must = 1                 pos : neg    =     14.6 : 1.0
                    flat = 1                 neg : pos    =     13.7 : 1.0
                loved_it = 1                 pos : neg    =     13.0 : 1.0
                    weak = 1                 neg : pos    =     13.0 : 1.0


**Validating**

In [13]:
refsets = nltk.collections.defaultdict(set)
testsets = nltk.collections.defaultdict(set)
real = []
result = []

for i, (feats, label) in enumerate(BoWFeatures):
    refsets[label].add(i)
    observed = classifier.classify(feats)
    testsets[observed].add(i)
    real.append(label)
    result.append(observed)

print( 'For Validating set:')
print( 'Accuracy:', nltk.accuracy(real,result) )
print( 'Precision:', nltk.scores.precision(refsets['pos'], testsets['pos']) )
print( 'Recall:', nltk.recall(refsets['pos'], testsets['pos']) )
print( 'F1 Score:', nltk.f_measure(refsets['pos'], testsets['pos']) )

For Validating set:
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0


**Testing**

In [14]:
refsets = nltk.collections.defaultdict(set)
testsets = nltk.collections.defaultdict(set)
real = []
result = []

for i, (feats, label) in enumerate(BoWTesting):
    refsets[label].add(i)
    observed = classifier.classify(feats)
    testsets[observed].add(i)
    real.append(label)
    result.append(observed)

print( 'For testing set:')
print( 'Accuracy:', nltk.accuracy(real,result) )
print( 'Precision:', nltk.scores.precision(refsets['pos'], testsets['pos']) )
print( 'Recall:', nltk.recall(refsets['pos'], testsets['pos']) )
print( 'F1 Score:', nltk.f_measure(refsets['pos'], testsets['pos']) )

For testing set:
Accuracy: 0.8199328107502799
Precision: 0.8907922912205567
Recall: 0.734982332155477
F1 Score: 0.8054211035818005


**BoW - Linear Regression**

Trains the dataset over a Linear Regression with a Bag of Words Model.

**Training**

In [15]:
shuffle(BoWFeatures)
classifier = nltk.classify.SklearnClassifier(LogisticRegression(max_iter=1000))
classifier.train(BoWFeatures)

<SklearnClassifier(LogisticRegression(max_iter=1000))>

**Validating**

In [16]:
refsets = nltk.collections.defaultdict(set)
testsets = nltk.collections.defaultdict(set)
real = []
result = []

for i, (feats, label) in enumerate(BoWFeatures):
    refsets[label].add(i)
    observed = classifier.classify(feats)
    testsets[observed].add(i)
    real.append(label)
    result.append(observed)

print( 'For Validating set:')
print( 'Accuracy:', nltk.accuracy(real,result) )
print( 'Precision:', nltk.scores.precision(refsets['pos'], testsets['pos']) )
print( 'Recall:', nltk.recall(refsets['pos'], testsets['pos']) )
print( 'F1 Score:', nltk.f_measure(refsets['pos'], testsets['pos']) )

For Validating set:
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0


**Testing**

In [17]:
refsets = nltk.collections.defaultdict(set)
testsets = nltk.collections.defaultdict(set)
real = []
result = []

for i, (feats, label) in enumerate(BoWTesting):
    refsets[label].add(i)
    observed = classifier.classify(feats)
    testsets[observed].add(i)
    real.append(label)
    result.append(observed)

print( 'For testing set:')
print( 'Accuracy:', nltk.accuracy(real,result) )
print( 'Precision:', nltk.scores.precision(refsets['pos'], testsets['pos']) )
print( 'Recall:', nltk.recall(refsets['pos'], testsets['pos']) )
print( 'F1 Score:', nltk.f_measure(refsets['pos'], testsets['pos']) )

For testing set:
Accuracy: 0.832474804031355
Precision: 0.8383928571428572
Recall: 0.8295053003533569
F1 Score: 0.8339253996447602


In [18]:
show_most_informative_features(classifier._vectorizer,classifier._clf,10)  

	-0.9262	bad            		1.0701	excellent      
	-0.8353	no             		0.8758	great          
	-0.8307	boring         		0.6934	loved          
	-0.7569	not            		0.6637	easy           
	-0.7350	waste          		0.5947	wonderful      
	-0.7229	too            		0.5774	enjoyed        
	-0.6363	worst          		0.5472	the_best       
	-0.6246	disappointing  		0.4820	a_little       
	-0.5940	don't          		0.4813	everything     
	-0.5754	if             		0.4762	best           


**Binary BoW - Naive Bayes**

Trains the dataset over a Naive Bayes with a Binary Bag of Words Model.

**Training**

In [19]:
shuffle(BinaryBoWFeatures)
classifier = nltk.NaiveBayesClassifier.train(BinaryBoWFeatures)
classifier.show_most_informative_features(10)

Most Informative Features
           disappointing = 1                 neg : pos    =     42.3 : 1.0
              your_money = 1                 neg : pos    =     28.3 : 1.0
              waste_your = 1                 neg : pos    =     21.7 : 1.0
                    weak = 1                 neg : pos    =     15.7 : 1.0
             a_wonderful = 1                 pos : neg    =     15.4 : 1.0
                  a_must = 1                 pos : neg    =     15.0 : 1.0
                    flat = 1                 neg : pos    =     15.0 : 1.0
               of_<year> = 1                 pos : neg    =     15.0 : 1.0
                  poorly = 1                 neg : pos    =     15.0 : 1.0
                   waste = 1                 neg : pos    =     14.6 : 1.0


**Validating**

In [20]:
refsets = nltk.collections.defaultdict(set)
testsets = nltk.collections.defaultdict(set)
real = []
result = []

for i, (feats, label) in enumerate(BinaryBoWFeatures):
    refsets[label].add(i)
    observed = classifier.classify(feats)
    testsets[observed].add(i)
    real.append(label)
    result.append(observed)

print( 'For Validating set:')
print( 'Accuracy:', nltk.accuracy(real,result) )
print( 'Precision:', nltk.scores.precision(refsets['pos'], testsets['pos']) )
print( 'Recall:', nltk.recall(refsets['pos'], testsets['pos']) )
print( 'F1 Score:', nltk.f_measure(refsets['pos'], testsets['pos']) )

For Validating set:
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0


**Testing**

In [21]:
refsets = nltk.collections.defaultdict(set)
testsets = nltk.collections.defaultdict(set)
real = []
result = []

for i, (feats, label) in enumerate(BinaryBoWTesting):
    refsets[label].add(i)
    observed = classifier.classify(feats)
    testsets[observed].add(i)
    real.append(label)
    result.append(observed)

print( 'For testing set:')
print( 'Accuracy:', nltk.accuracy(real,result) )
print( 'Precision:', nltk.scores.precision(refsets['pos'], testsets['pos']) )
print( 'Recall:', nltk.recall(refsets['pos'], testsets['pos']) )
print( 'F1 Score:', nltk.f_measure(refsets['pos'], testsets['pos']) )

For testing set:
Accuracy: 0.8219484882418813
Precision: 0.8968125337655322
Recall: 0.7332155477031802
F1 Score: 0.8068043742405832


**Binary BoW - Linear Regression**

Trains the dataset over a Linear Regression with a Binary Bag of Words Model.

**Training**

In [22]:
shuffle(BinaryBoWFeatures)
classifier = nltk.classify.SklearnClassifier(LogisticRegression(max_iter=1000))
classifier.train(BinaryBoWFeatures)

<SklearnClassifier(LogisticRegression(max_iter=1000))>

**Validating**

In [23]:
refsets = nltk.collections.defaultdict(set)
testsets = nltk.collections.defaultdict(set)
real = []
result = []

for i, (feats, label) in enumerate(BinaryBoWFeatures):
    refsets[label].add(i)
    observed = classifier.classify(feats)
    testsets[observed].add(i)
    real.append(label)
    result.append(observed)

print( 'For Validating set:')
print( 'Accuracy:', nltk.accuracy(real,result) )
print( 'Precision:', nltk.scores.precision(refsets['pos'], testsets['pos']) )
print( 'Recall:', nltk.recall(refsets['pos'], testsets['pos']) )
print( 'F1 Score:', nltk.f_measure(refsets['pos'], testsets['pos']) )

For Validating set:
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0


**Testing**

In [24]:
refsets = nltk.collections.defaultdict(set)
testsets = nltk.collections.defaultdict(set)
real = []
result = []

for i, (feats, label) in enumerate(BinaryBoWTesting):
    refsets[label].add(i)
    observed = classifier.classify(feats)
    testsets[observed].add(i)
    real.append(label)
    result.append(observed)

print( 'For testing set:')
print( 'Accuracy:', nltk.accuracy(real,result) )
print( 'Precision:', nltk.scores.precision(refsets['pos'], testsets['pos']) )
print( 'Recall:', nltk.recall(refsets['pos'], testsets['pos']) )
print( 'F1 Score:', nltk.f_measure(refsets['pos'], testsets['pos']) )

For testing set:
Accuracy: 0.8369540873460246
Precision: 0.8410301953818827
Recall: 0.8365724381625441
F1 Score: 0.8387953941541186


In [25]:
show_most_informative_features(classifier._vectorizer,classifier._clf,10)  

	-1.1242	not            		1.0857	excellent      
	-0.8812	no             		0.9634	great          
	-0.8507	boring         		0.6946	loved          
	-0.8445	bad            		0.6809	wonderful      
	-0.8166	waste          		0.6527	the_best       
	-0.6822	disappointing  		0.6424	easy           
	-0.6651	don't          		0.5726	enjoyed        
	-0.6595	too            		0.5509	a_little       
	-0.6359	worst          		0.5078	best           
	-0.6073	didn't         		0.4844	must           


**Lexicon Features - Naive Bayes**

Trains the dataset over a Naive Bayes with a Lexicon Features Model.

**Training**

In [26]:
shuffle(LexiconFeatures)
classifier = nltk.NaiveBayesClassifier.train(LexiconFeatures)
classifier.show_most_informative_features(10)

Most Informative Features
               count_pos = 13                pos : neg    =      6.3 : 1.0
               point_neg = 8                 neg : pos    =      5.7 : 1.0
               count_neg = 11                neg : pos    =      5.2 : 1.0
               count_neg = 17                neg : pos    =      5.2 : 1.0
               count_neg = 10                neg : pos    =      5.0 : 1.0
               count_pos = 22                pos : neg    =      5.0 : 1.0
                    size = 96                neg : pos    =      5.0 : 1.0
               point_pos = 22                pos : neg    =      5.0 : 1.0
               point_pos = 49                pos : neg    =      5.0 : 1.0
               point_pos = 81                pos : neg    =      4.7 : 1.0


**Validating**

In [27]:
refsets = nltk.collections.defaultdict(set)
testsets = nltk.collections.defaultdict(set)
real = []
result = []

for i, (feats, label) in enumerate(LexiconFeatures):
    refsets[label].add(i)
    observed = classifier.classify(feats)
    testsets[observed].add(i)
    real.append(label)
    result.append(observed)

print( 'For Validating set:')
print( 'Accuracy:', nltk.accuracy(real,result) )
print( 'Precision:', nltk.scores.precision(refsets['pos'], testsets['pos']) )
print( 'Recall:', nltk.recall(refsets['pos'], testsets['pos']) )
print( 'F1 Score:', nltk.f_measure(refsets['pos'], testsets['pos']) )

For Validating set:
Accuracy: 0.91
Precision: 0.9019607843137255
Recall: 0.92
F1 Score: 0.9108910891089109


**Testing**

In [28]:
refsets = nltk.collections.defaultdict(set)
testsets = nltk.collections.defaultdict(set)
real = []
result = []

for i, (feats, label) in enumerate(LexiconTesting):
    refsets[label].add(i)
    observed = classifier.classify(feats)
    testsets[observed].add(i)
    real.append(label)
    result.append(observed)

print( 'For testing set:')
print( 'Accuracy:', nltk.accuracy(real,result) )
print( 'Precision:', nltk.scores.precision(refsets['pos'], testsets['pos']) )
print( 'Recall:', nltk.recall(refsets['pos'], testsets['pos']) )
print( 'F1 Score:', nltk.f_measure(refsets['pos'], testsets['pos']) )

For testing set:
Accuracy: 0.6472564389697648
Precision: 0.6573777980813157
Recall: 0.6356007067137809
F1 Score: 0.6463058612171569


**Lexicon Features - Linear Regression**

Trains the dataset over a Linear Regression with a Lexicon Features Model.

**Training**

In [29]:
shuffle(LexiconFeatures)
classifier = nltk.classify.SklearnClassifier(LogisticRegression(max_iter=1000))
classifier.train(LexiconFeatures)

<SklearnClassifier(LogisticRegression(max_iter=1000))>

**Validating**

In [30]:
refsets = nltk.collections.defaultdict(set)
testsets = nltk.collections.defaultdict(set)
real = []
result = []

for i, (feats, label) in enumerate(LexiconFeatures):
    refsets[label].add(i)
    observed = classifier.classify(feats)
    testsets[observed].add(i)
    real.append(label)
    result.append(observed)

print( 'For Validating set:')
print( 'Accuracy:', nltk.accuracy(real,result) )
print( 'Precision:', nltk.scores.precision(refsets['pos'], testsets['pos']) )
print( 'Recall:', nltk.recall(refsets['pos'], testsets['pos']) )
print( 'F1 Score:', nltk.f_measure(refsets['pos'], testsets['pos']) )

For Validating set:
Accuracy: 0.692
Precision: 0.6935483870967742
Recall: 0.688
F1 Score: 0.6907630522088354


**Testing**

In [31]:
refsets = nltk.collections.defaultdict(set)
testsets = nltk.collections.defaultdict(set)
real = []
result = []

for i, (feats, label) in enumerate(LexiconTesting):
    refsets[label].add(i)
    observed = classifier.classify(feats)
    testsets[observed].add(i)
    real.append(label)
    result.append(observed)

print( 'For testing set:')
print( 'Accuracy:', nltk.accuracy(real,result) )
print( 'Precision:', nltk.scores.precision(refsets['pos'], testsets['pos']) )
print( 'Recall:', nltk.recall(refsets['pos'], testsets['pos']) )
print( 'F1 Score:', nltk.f_measure(refsets['pos'], testsets['pos']) )

For testing set:
Accuracy: 0.677043673012318
Precision: 0.6801051709027169
Recall: 0.6855123674911661
F1 Score: 0.6827980642322922


In [32]:
show_most_informative_features(classifier._vectorizer,classifier._clf,10)  

	-3.7594	porc_neg       		4.4370	porc_pos       
	-0.0673	count_pos      		0.0426	point_pos      
	-0.0578	count_neg      		0.0083	point_neg      
	0.0013	size           		0.0013	size           
	0.0083	point_neg      		-0.0578	count_neg      
	0.0426	point_pos      		-0.0673	count_pos      
	4.4370	porc_pos       		-3.7594	porc_neg       


**Table of Results - Books**

| Combination | Accuracy | Precision | Recall | F1 Score|
| --- | ----------- |----------- |----------- |----------- |
| **BOW - NB** | 0.8199328107502799 | 0.8907922912205567 | 0.734982332155477 | 0.8054211035818005 |
| **BOW - LR** | 0.832474804031355 | 0.8383928571428572 | 0.8295053003533569 | 0.8339253996447602 |
| **BBOW - NB** | 0.8219484882418813 | **0.8968125337655322** | 0.7332155477031802 | 0.8068043742405832 |
| **BBOW - LR** | **0.8369540873460246** | 0.8410301953818827 | **0.8365724381625441** | **0.8387953941541186** |
| **Lexicon - NB** | 0.6472564389697648 | 0.6573777980813157 | 0.6356007067137809 | 0.6463058612171569 |
| **Lexicon - LR** | 0.677043673012318 | 0.6801051709027169 | 0.6855123674911661 | 0.6827980642322922 |